## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
# this is the launching of mlflow without connecting it to any server. this is when you are doing it for a client and it is not a collaborative project.
import mlflow

In [2]:
# this is the code to ask mlflow to print me the backend I will be using as a personal developer

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///workspaces/mlops-afrimcamp/02-tracking-experiment/running-mlflow-examples/mlruns'


In [3]:
mlflow.list_experiments()

AttributeError: module 'mlflow' has no attribute 'list_experiments'

You will realised we have an error while funning this mlflow.list_experiments(). This is because the error indicates that the list_experiments function has been deprecated in recent versions of MLflow (likely since version 1.9 or later). The recommended approach to list experiments is now using the search_experiments function.

Solution:

1) use search_experiments:

mlflow.search_experiments()

2) print search_experiments

experiments = search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Experiment Name: {experiment.name}")
    
Access other experiment attributes like creation_time, lifecycle_stage, etc.


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-afrimcamp/02-tracking%20experiment/running-mlflow-examples/mlruns/945630649382285031', creation_time=1719056365143, experiment_id='945630649382285031', last_update_time=1719056365143, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/mlops-afrimcamp/02-tracking%20experiment/running-mlflow-examples/mlruns/0', creation_time=1719055980451, experiment_id='0', last_update_time=1719055980451, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1") # this is the new experiment we created since mlflow created a default one for us.

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'file:///workspaces/mlops-afrimcamp/02-tracking%20experiment/running-mlflow-examples/mlruns/945630649382285031/a05eb8dc87184090b422dd4ccff7690e/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-afrimcamp/02-tracking%20experiment/running-mlflow-examples/mlruns/945630649382285031', creation_time=1719056365143, experiment_id='945630649382285031', last_update_time=1719056365143, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/mlops-afrimcamp/02-tracking%20experiment/running-mlflow-examples/mlruns/0', creation_time=1719055980451, experiment_id='0', last_update_time=1719055980451, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [8]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")